In [457]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [458]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [459]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [460]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [461]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [462]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [463]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.215s

OK


In [464]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [465]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [466]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mhREsddRNQ4i4qYi2cH764b7RFxaiu5ar8


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [467]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.169s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [468]:
# Exercise 4

from ecc import PrivateKey
from helper import hash256, little_endian_to_int
from script import p2wpkh_script, Script
from tx import TxIn, TxOut, Tx
import bech32

def decode_bech32(addr):
    """ Decode a Bech32 address and return the witness program. """
    hrp, data = bech32.bech32_decode(addr)
    if hrp is None or data is None:
        raise ValueError("Invalid Bech32 address")
    decoded_data = bytes(bech32.convertbits(data[1:], 5, 8, False))
    if not decoded_data:
        raise ValueError("Error in Bech32 conversion")
    return decoded_data

# Private key setup using a passphrase
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)

# Print the private key in integer form
print("Private key (integer):", priv.secret)

# UTXO details
prev_tx = bytes.fromhex('6e08da198fbb9f6e682fd09240fd9a7d3a5e35f3836b60b2315e8b57ca7319d8')
prev_index = 0
target_address = 'tb1qerzrlxcfu24davlur5sqmgzzgsal6wusda40er'
change_address = 'tb1qzndeshkr5xsdh7m8ks9s0g6cy2966gfd6u5637'
prev_tx_amount = 0.00013711  # BTC
fee = 0.00000185  # BTC

# Calculating the amounts for the transaction outputs
send_amount = int(prev_tx_amount * 0.6 * 1e8)  # 60% to the target address
change_amount = int((prev_tx_amount - send_amount / 1e8 - fee) * 1e8)  # Remaining amount minus fees back as change

# Transaction inputs and outputs setup
tx_ins = [TxIn(prev_tx, prev_index)]
tx_outs = []

# Output to the target address (P2WPKH)
decoded_data = decode_bech32(target_address)
script_pubkey = p2wpkh_script(decoded_data)
tx_outs.append(TxOut(send_amount, script_pubkey))

# Output to the change address (P2WPKH)
decoded_data = decode_bech32(change_address)
script_pubkey = p2wpkh_script(decoded_data)
tx_outs.append(TxOut(change_amount, script_pubkey))

# Creating and signing the transaction
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
if tx_obj.sign_input(0, priv):
    print("Input signed successfully.")
else:
    print("Input signing failed.")

# Serializing the transaction
serialized_tx = tx_obj.serialize().hex()
print("Serialized Transaction:", serialized_tx)




# The transaction can now be broadcasted to a testnet blockchain explorer


Private key (integer): 72126807206263466701712740363329084999950309380954694321123174482932578356338
Input signed successfully.
Serialized Transaction: 0100000001d81973ca578b5e31b2606b83f3355e3a7d9afd4092d02f686e9fbb8f19da086e000000006a47304402204ae790abeb648f531c3013c48ce544bec32b3aa4f7be454da3337588a01829ce0220328f4f2334dcb6bddb42a8eaf99b4ff02153a57d7c7a0ccee6df3cc8a38fac9801210223f2d2de69a1a472aab865cd6a88cb7f848b4777f197f66699baa500d111b2e0ffffffff022220000000000000160014c8c43f9b09e2aadeb3fc1d200da042443bfd3b90b31400000000000016001414db985ec3a1a0dbfb67b40b07a358228bad212d00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [469]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

def bech32_decode(addr):
    import bech32
    hrp, data = bech32.bech32_decode(addr)
    if hrp is None or data is None:
        raise ValueError("Invalid Bech32 address")
    decoded_data = bytes(bech32.convertbits(data[1:], 5, 8, False))
    return decoded_data

# Provided from Exercise 4
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)

# Details for the first input (from a testnet faucet transaction)
prev_tx1 = bytes.fromhex('e3b0c44298fc1c149afbf4c8996f92427ae41e4649b934ca495991b7852b855')
prev_index1 = 0  # Index of the output in the previous transaction we want to spend

# Details for the second input (from Exercise 4 transaction)
prev_tx2 = bytes.fromhex('6e08da198fbb9f6e682fd09240fd9a7d3a5e35f3836b60b2315e8b57ca7319d8')
prev_index2 = 0

# Transaction inputs
tx_ins = [
    TxIn(prev_tx1, prev_index1),  # First input
    TxIn(prev_tx2, prev_index2)   # Second input
]

# Total input amount sum from the testnet faucet and the previous exercise
input_amount = 0.001 + 0.00013711  # Just an example, replace with actual sums

# Transaction output details
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
fee = 0.00001000  # Transaction fee
output_amount = int((input_amount - fee) * 1e8)

# Transaction output
tx_outs = [
    TxOut(output_amount, script_pubkey)
]

# Create the transaction
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# Sign the first input
if tx_obj.sign_input(0, priv, SIGHASH_ALL):
    print("First input signed successfully.")
else:
    print("Failed to sign the first input.")

# Sign the second input
if tx_obj.sign_input(1, priv, SIGHASH_ALL):
    print("Second input signed successfully.")
else:
    print("Failed to sign the second input.")

# Serialize the transaction
serialized_tx = tx_obj.serialize().hex()
print("Serialized Transaction:", serialized_tx)

import requests
url = "https://api.blockcypher.com/v1/btc/test3/txs/push"
response = requests.post(url, json={'tx': serialized_tx})
print(response.json())



ValueError: non-hexadecimal number found in fromhex() arg at position 63

In [470]:
# Exercize 6: Get proper testnet Bech32 change_address

from ecc import PrivateKey
from helper import hash256, little_endian_to_int, hash160
import bech32

def bech32_address(pubkey, testnet=True):
    """ Generate a Bech32 address from a public key """
    # Compute the Witness Program (HASH160 for P2WPKH)
    wp = hash160(pubkey.sec(compressed=True))
    
    # Bech32 human-readable part (hrp) differs for testnet and mainnet
    hrp = 'tb' if testnet else 'bc'
    
    # Convert the 20-byte witness program (version 0 for P2WPKH) to base32
    data = bech32.convertbits(wp, 8, 5)
    
    # Encode as bech32 and return the address
    return bech32.bech32_encode(hrp, [0] + data)

# Usage:
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)

# Generate the Bech32 address
address = bech32_address(private_key.point, testnet=True)
print("Bech32 Address:", address)

Bech32 Address: tb1qzndeshkr5xsdh7m8ks9s0g6cy2966gfd6u5637


In [471]:
# Exercize 7:

from ecc import PrivateKey
from helper import hash256, little_endian_to_int
from mnemonic import Mnemonic

def generate_mnemonic(entropy_bytes):
    mnemo = Mnemonic("english")
    mnemonic_words = mnemo.to_mnemonic(entropy_bytes)
    return mnemonic_words

# Generating private key using a passphrase
passphrase = b'brjf@yahoo.com I did not make it all the way through 3rd grade for nothin'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)

# Display the private key as an integer
print("Private key (integer):", priv.secret)

# Prepare entropy by taking the first 128 bits of the private key
# We use 32 bytes because hash256 returns a 256-bit number
entropy_bytes = priv.secret.to_bytes(32, 'big')[:16]  # Get the first 16 bytes (128 bits)
mnemonic = generate_mnemonic(entropy_bytes)
print("Mnemonic:", mnemonic)


Private key (integer): 72126807206263466701712740363329084999950309380954694321123174482932578356338
Mnemonic: panda raw theory general tonight drive pill cheap deposit whisper use learn
